# Segment raw images to get gray matter before normalising.
This is to prevent errors in normalisation down the line (e.g. enlargement of ventricles: GM WM growth or shrinkage?)

In [1]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.spm as spm
import numpy as np
import pandas as pd
import os, re, random


200122-14:21:11,947 nipype.utils INFO:
	 Running nipype version 1.4.0-dev (latest: 1.4.0)


In [4]:
# define useful functions

# get subject directories from parent directory
def get_subdirs(dir):
    subdirs=[]
    for i in os.listdir(dir):
        # verify subject number in directory and exclude other dirs
        match=re.match(r'(.*\d{3}$)', i)
        if os.path.isdir(os.path.join(dir, i)) and match:
            subdirs.append(os.path.join(dir, match.group(0)))
    return sorted(subdirs)

In [5]:
subdirs=get_subdirs("/VBM/")
print(subdirs)

['/VBM/FED006', '/VBM/FED007', '/VBM/FED008', '/VBM/FED009', '/VBM/FED010', '/VBM/FED011', '/VBM/FED012', '/VBM/FED013', '/VBM/FED014', '/VBM/FED015', '/VBM/FED016', '/VBM/FED017', '/VBM/FED018', '/VBM/FED019', '/VBM/FED020', '/VBM/FED021', '/VBM/FED022', '/VBM/FED023', '/VBM/FED024', '/VBM/FED025', '/VBM/FED026', '/VBM/FED027', '/VBM/FED028', '/VBM/FED029', '/VBM/FED030', '/VBM/FED031', '/VBM/FED032', '/VBM/FED033', '/VBM/FED034', '/VBM/FED035', '/VBM/FED036', '/VBM/FED037', '/VBM/FED038', '/VBM/FED039', '/VBM/FED040', '/VBM/FED041', '/VBM/FED042', '/VBM/FED043', '/VBM/FED044', '/VBM/FED045', '/VBM/FED046', '/VBM/FED047', '/VBM/FED048', '/VBM/FED049', '/VBM/FED050', '/VBM/FED051', '/VBM/FED052', '/VBM/FED053', '/VBM/FED054', '/VBM/FED055', '/VBM/FED056', '/VBM/FED057', '/VBM/FED058', '/VBM/FED059', '/VBM/FED060', '/VBM/FED061', '/VBM/FED062', '/VBM/FED063', '/VBM/FED064', '/VBM/FED065', '/VBM/FED066', '/VBM/FED067', '/VBM/FED068']


In [6]:
# make dict with all T1-images as pathlike objects
images_T1={}
for i in subdirs:
    for j in os.listdir(i):
        match=re.match(r'(.*T1_MPRAGE.*\d{14,}.*.nii)', j)
        if match:
            images_T1[i]=os.path.join(i, match.group(0))

print(images_T1)

{'/VBM/FED006': '/VBM/FED006/T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429_5.nii', '/VBM/FED007': '/VBM/FED007/T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226_5.nii', '/VBM/FED008': '/VBM/FED008/T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009_5.nii', '/VBM/FED009': '/VBM/FED009/T1_MPRAGE_SAG_ISO_0_9_0005_20141117154954_5.nii', '/VBM/FED010': '/VBM/FED010/T1_MPRAGE_SAG_ISO_0_9_0005_20141119091904_5.nii', '/VBM/FED011': '/VBM/FED011/T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708_5.nii', '/VBM/FED012': '/VBM/FED012/T1_MPRAGE_SAG_ISO_0_9_0005_20141124150148_5.nii', '/VBM/FED013': '/VBM/FED013/T1_MPRAGE_SAG_ISO_0_9_0005_20141125093955_5.nii', '/VBM/FED014': '/VBM/FED014/T1_MPRAGE_SAG_ISO_0_9_0005_20141125153216_5.nii', '/VBM/FED015': '/VBM/FED015/T1_MPRAGE_SAG_ISO_0_9_0005_20141202145526_5.nii', '/VBM/FED016': '/VBM/FED016/T1_MPRAGE_SAG_ISO_0_9_0005_20141202164821_5.nii', '/VBM/FED017': '/VBM/FED017/T1_MPRAGE_SAG_ISO_0_9_0005_20141204151043_5.nii', '/VBM/FED018': '/VBM/FED018/T1_MPRAGE_SAG_ISO_0_9_0005_20141208

In [8]:
# use NewSegment from spm to segment unimportant anatomical landmarks and perform "brain-extraction" (TEST FED006)
#for i in images.keys(): use this when ready to segment full subject list

# classify tissues from tpm.nii
# need to perform segmentation with full tissue information, or brain will be classified as "soft tissue" etc. -> see examples in "/VBM/FED006/"
tissue1 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 1), 2, (True,True), (False, False)) # GM
tissue2 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 2), 2, (True,True), (False, False)) # WM
tissue3 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 3), 2, (True,True), (False, False)) # CSF
tissue4 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 4), 2, (False,False), (False, False)) # bone
tissue5 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 5), 2, (False,False), (False, False)) # soft tissue
tissue6 = (('/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii', 6), 2, (False,False), (False, False)) # air and background

# create SKULLSTRIP segment nodes for each subject
for i in images_T1.keys():
    segm_skullstrip_T1=Node(spm.preprocess.NewSegment(), name='segm_skullstrip_T1')
    segm_skullstrip_T1.inputs.tissues=[tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]
    # do not generate SPM8 and higher compatible jobs
    segm_skullstrip_T1.inputs.use_v8struct=False
    segm_skullstrip_T1.inputs.affine_regularization='none'
    # write forward and inverse deformation fields
    #segm_skullstrip_T1.inputs.write_deformation_fields=[True, True]
    # establish input files from subject
    segm_skullstrip_T1.inputs.channel_files=images_T1[i]
    # establish basedir for output of subject node
    for j in subdirs:
        if j.rsplit('/', 1)[1]==i:
            segm_skullstrip_T1.base_dir=j
    results=segm_skullstrip_T1.run()
    print(results.outputs)

200109-12:58:20,555 nipype.workflow INFO:
	 [Node] Setting-up "segm_skullstrip_T1" in "/VBM/FED009/segm_skullstrip_T1".
200109-12:58:20,562 nipype.workflow INFO:
	 [Node] Running "segm_skullstrip_T1" ("nipype.interfaces.spm.preprocess.NewSegment")
200109-13:02:05,551 nipype.workflow INFO:
	 [Node] Finished "segm_skullstrip_T1".

bias_corrected_images = <undefined>
bias_field_images = <undefined>
dartel_input_images = [['/VBM/FED009/segm_skullstrip_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141117154954_5.nii'], ['/VBM/FED009/segm_skullstrip_T1/rc2T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141117154954_5.nii'], ['/VBM/FED009/segm_skullstrip_T1/rc3T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141117154954_5.nii'], [], [], []]
forward_deformation_field = <undefined>
inverse_deformation_field = <undefined>
modulated_class_images = [[], [], [], [], [], []]
native_class_images = [['/VBM/FED009/segm_skullstrip_T1/c1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20

In [7]:
# get segmentation directory for each subject

# define start of segmentation directory name
targetnode="segm"
taskdirs={}

for i in images_T1.keys():
    for j in subdirs:
        # match keys with subdir list
        if j == i: 
            for k in os.listdir(j):
                # go through content of subdirs and look for "true" directories
                if os.path.isdir(os.path.join(j, k)):
                    # regex functions match=exact match of pattern, search=finds elements that contain pattern
                    # regex matches entries in j with targetnode k
                    match=re.match(r'(' + targetnode + ').*', k)
                    # ? This is needed to recognize the attribute group from the regex output
                    if match: 
                        taskdirs[j]=os.path.join(j, match.group(0))

create GM template using SPM DARTEL

In [8]:
# split taskdirs in control and depressed

# define and load from .xls(x) files
sub_control_ID="FED_Subject_Assignments.xlsx"
control_ID=pd.read_excel(sub_control_ID, sheet_name="Sheet2").astype(str)
ID=control_ID['Sub Num FED_XXX'].str.zfill(3).tolist()
status=control_ID['0 = Control'].tolist()

control_taskdir={}
depress_taskdir={}

for i in taskdirs.keys():
    for x,y in zip(ID,status):
        if i.rsplit('/',1)[1] == "FED"+x and y == "0":
            control_taskdir[i]=taskdirs[i]

for i in taskdirs.keys():
    if i not in control_taskdir.keys():
        depress_taskdir[i]=taskdirs[i]

# check dicts
print(control_taskdir.keys())
print(depress_taskdir.keys())
# see, if sizes are about equal
print(len(control_taskdir.keys()), len(depress_taskdir.keys()))

dict_keys(['/VBM/FED006', '/VBM/FED007', '/VBM/FED008', '/VBM/FED009', '/VBM/FED012', '/VBM/FED013', '/VBM/FED015', '/VBM/FED016', '/VBM/FED017', '/VBM/FED019', '/VBM/FED020', '/VBM/FED021', '/VBM/FED022', '/VBM/FED023', '/VBM/FED024', '/VBM/FED025', '/VBM/FED026', '/VBM/FED049', '/VBM/FED053', '/VBM/FED054', '/VBM/FED056', '/VBM/FED058', '/VBM/FED060', '/VBM/FED061', '/VBM/FED062', '/VBM/FED063', '/VBM/FED064', '/VBM/FED067', '/VBM/FED068'])
dict_keys(['/VBM/FED010', '/VBM/FED011', '/VBM/FED014', '/VBM/FED018', '/VBM/FED027', '/VBM/FED028', '/VBM/FED029', '/VBM/FED030', '/VBM/FED031', '/VBM/FED032', '/VBM/FED033', '/VBM/FED034', '/VBM/FED035', '/VBM/FED036', '/VBM/FED037', '/VBM/FED038', '/VBM/FED039', '/VBM/FED040', '/VBM/FED041', '/VBM/FED042', '/VBM/FED043', '/VBM/FED044', '/VBM/FED045', '/VBM/FED046', '/VBM/FED047', '/VBM/FED048', '/VBM/FED050', '/VBM/FED051', '/VBM/FED052', '/VBM/FED055', '/VBM/FED057', '/VBM/FED059', '/VBM/FED065', '/VBM/FED066'])
29 34


In [7]:
# create template dict
# collect random keys (29 from each group)

# only 29 controls, so take all into template
template_dirs=control_taskdir.copy()
keys=random.sample(list(depress_taskdir), 29)
# integrate both dicts for template_dirs
for i in keys:
    template_dirs[i]=depress_taskdir[i]

# get rc1 and rc2 files as path-like objects
rc1=[]
rc2=[]
# append files to lists
for k,v in template_dirs.items():
    for i in os.listdir(v):
        # look for rc1(GM)
        match1=re.match(r'(^rc1.*.nii)', i)
        if match1:
            rc1.append(os.path.join(v, match1.group(0)))
        # look for rc2(WM)
        match2=re.match(r'(^rc2.*.nii)', i)
        if match2:
            rc2.append(os.path.join(v, match2.group(0)))
# print lists
print(rc1, len(rc1))
print(rc2, len(rc2))

['/VBM/FED006/segm_skullstrip_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii', '/VBM/FED007/segm_skullstrip_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141107104226_5.nii', '/VBM/FED008/segm_skullstrip_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141112155009_5.nii', '/VBM/FED009/segm_skullstrip_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141117154954_5.nii', '/VBM/FED012/segm_skullstrip_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141124150148_5.nii', '/VBM/FED013/segm_skullstrip_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141125093955_5.nii', '/VBM/FED015/segm_skullstrip_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141202145526_5.nii', '/VBM/FED016/segm_skullstrip_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141202164821_5.nii', '/VBM/FED017/segm_skullstrip_T1/rc1T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141204151043_5.nii', '/VBM/FED019/segm_skullstrip_T1/rc1T

In [8]:
# create DARTEL node and template
dartel_template=Node(spm.DARTEL(), name='dartel_template')
dartel_template.inputs.image_files=[rc1, rc2]
dartel_template.inputs.template_prefix="FED_ConDepr-grouptemplate"
dartel_template.base_dir="/VBM"
# do not generate SPM8 and higher compatible jobs
dartel_template.inputs.use_v8struct=False
results=dartel_template.run()

200114-15:39:53,669 nipype.workflow INFO:
	 [Node] Setting-up "dartel_template" in "/VBM/dartel_template".
200114-15:39:53,705 nipype.workflow INFO:
	 [Node] Running "dartel_template" ("nipype.interfaces.spm.preprocess.DARTEL")
200114-18:02:43,883 nipype.workflow INFO:
	 [Node] Finished "dartel_template".


normalise to MNI space

In [11]:
# get required file lists and check that theyre in the same order

# deformation flowfields from nonlinear registration
flowfields=[]
# GM and WM 
c1=[]
c2=[]

# append files to lists
# flowfields
finaltemplate_dir="/VBM/dartel_template/"
for i in os.listdir(finaltemplate_dir):
    # look for u_rc1
    match=re.match(r'(^u_rc1.*.nii)', i)
    if match:
        flowfields.append(os.path.join(finaltemplate_dir, match.group(0)))

# check flowfields against subject T1 images to see 
# which were picked to create the template
flowfields_matched={}
for k,v in images_T1.items():
    for i in flowfields:
        # if cropped u_rc1T1 and T1 overlap via sequence and reg-number,
        # associate FEDXXX to flowfields-value
            if i.rsplit('/',1)[1][5:48] == v.rsplit('/',1)[1][0:43]:
                flowfields_matched[k]=i

# bring matches into sorted order for final list
flowfields=[]
for k,v in flowfields_matched.items():
    flowfields.append(v)
    

# GM and WM
# create dict where flowfields_matched and taskdirs(segmentation) overlap
intemplate_dirs={}
for i in flowfields_matched.keys():
    for k,v in taskdirs.items():
        if i == k:
            intemplate_dirs[i]=v

# fill c1 and c2 lists
for k,v in intemplate_dirs.items():
    for i in os.listdir(v):
        # look for c1(GM)
        match1=re.match(r'(^c1.*.nii)', i)
        if match1:
            c1.append(os.path.join(v, match1.group(0)))
        # look for c2(WM)
        match2=re.match(r'(^c2.*.nii)', i)
        if match2:
            c2.append(os.path.join(v, match2.group(0)))

In [15]:
# check flowfields_matched
print(flowfields_matched.keys())

dict_keys(['/VBM/FED006', '/VBM/FED007', '/VBM/FED008', '/VBM/FED009', '/VBM/FED011', '/VBM/FED012', '/VBM/FED013', '/VBM/FED014', '/VBM/FED015', '/VBM/FED016', '/VBM/FED017', '/VBM/FED018', '/VBM/FED019', '/VBM/FED020', '/VBM/FED021', '/VBM/FED022', '/VBM/FED023', '/VBM/FED024', '/VBM/FED025', '/VBM/FED026', '/VBM/FED027', '/VBM/FED028', '/VBM/FED029', '/VBM/FED030', '/VBM/FED031', '/VBM/FED032', '/VBM/FED033', '/VBM/FED034', '/VBM/FED035', '/VBM/FED036', '/VBM/FED037', '/VBM/FED038', '/VBM/FED039', '/VBM/FED040', '/VBM/FED042', '/VBM/FED043', '/VBM/FED045', '/VBM/FED046', '/VBM/FED048', '/VBM/FED049', '/VBM/FED050', '/VBM/FED051', '/VBM/FED053', '/VBM/FED054', '/VBM/FED055', '/VBM/FED056', '/VBM/FED057', '/VBM/FED058', '/VBM/FED059', '/VBM/FED060', '/VBM/FED061', '/VBM/FED062', '/VBM/FED063', '/VBM/FED064', '/VBM/FED065', '/VBM/FED066', '/VBM/FED067', '/VBM/FED068'])


In [12]:
# check lists order
print(flowfields[0:3])
print(c1[0:3])
print(c2[0:3])

['/VBM/dartel_template/u_rc1T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429_5_FED_ConDepr-grouptemplate.nii', '/VBM/dartel_template/u_rc1T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226_5_FED_ConDepr-grouptemplate.nii', '/VBM/dartel_template/u_rc1T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009_5_FED_ConDepr-grouptemplate.nii']
['/VBM/FED006/segm_skullstrip_T1/c1T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429_5.nii', '/VBM/FED007/segm_skullstrip_T1/c1T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226_5.nii', '/VBM/FED008/segm_skullstrip_T1/c1T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009_5.nii']
['/VBM/FED006/segm_skullstrip_T1/c2T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429_5.nii', '/VBM/FED007/segm_skullstrip_T1/c2T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226_5.nii', '/VBM/FED008/segm_skullstrip_T1/c2T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009_5.nii']


In [68]:
# create DARTEL node to norm2MNI for Gm and WM
tissues={"c1": c1, "c2": c2}
for k,v in tissues.items():
    dartel_norm2mni=Node(spm.DARTELNorm2MNI(), name='dartel_norm2mni_'+str(k))
    dartel_norm2mni.inputs.template_file =str(finaltemplate_dir)+"FED_ConDepr-grouptemplate_6.nii"
    dartel_norm2mni.inputs.flowfield_files = flowfields
    dartel_norm2mni.inputs.apply_to_files = v
    dartel_norm2mni.inputs.modulate = True
    dartel_norm2mni.base_dir="/VBM"
    # think about smoothing kernel below ... ROI sizes?
    #dartel_norm2mni.inputs.fwhm =
    # do not generate SPM8 and higher compatible jobs
    dartel_norm2mni.inputs.use_v8struct=False
    results=dartel_norm2mni.run()

200115-16:22:19,251 nipype.workflow INFO:
	 [Node] Setting-up "dartel_norm2mni_c1" in "/VBM/dartel_norm2mni_c1".
200115-16:22:19,278 nipype.workflow INFO:
	 [Node] Running "dartel_norm2mni_c1" ("nipype.interfaces.spm.preprocess.DARTELNorm2MNI")
200115-16:28:35,843 nipype.workflow INFO:
	 [Node] Finished "dartel_norm2mni_c1".
200115-16:28:35,850 nipype.workflow INFO:
	 [Node] Setting-up "dartel_norm2mni_c2" in "/VBM/dartel_norm2mni_c2".
200115-16:28:35,875 nipype.workflow INFO:
	 [Node] Running "dartel_norm2mni_c2" ("nipype.interfaces.spm.preprocess.DARTELNorm2MNI")
200115-16:34:48,271 nipype.workflow INFO:
	 [Node] Finished "dartel_norm2mni_c2".


Testing Zone:

In [36]:
# check lists order (function^^)
regex=re.compile('\d{14}')
for i in list(flowfields,c1,c2):
    nums=list(filter(regex.match, i))
    

<built-in method groupdict of re.Match object at 0x7fccfc286f30>
<built-in method groupdict of re.Match object at 0x7fccfc128990>
<built-in method groupdict of re.Match object at 0x7fccfc128a80>
<built-in method groupdict of re.Match object at 0x7fccfc1287b0>
<built-in method groupdict of re.Match object at 0x7fccfc0d7c60>
<built-in method groupdict of re.Match object at 0x7fccfc0d7e40>
<built-in method groupdict of re.Match object at 0x7fccfc0d76c0>
<built-in method groupdict of re.Match object at 0x7fccfc0d7be8>
<built-in method groupdict of re.Match object at 0x7fccfc0d7d50>
<built-in method groupdict of re.Match object at 0x7fccfbdc4648>
<built-in method groupdict of re.Match object at 0x7fccfbdc4558>
<built-in method groupdict of re.Match object at 0x7fccfbdc46c0>
<built-in method groupdict of re.Match object at 0x7fccfbdc43f0>
<built-in method groupdict of re.Match object at 0x7fccfbdc4120>
<built-in method groupdict of re.Match object at 0x7fccfbdc40a8>
<built-in method groupdic

In [12]:
# crop down images for testing (only FED006,07,08!!!)
images2=dict()
for i in sorted(images.keys())[0:3]:
    images2[i]=images[i]
    
print(images2.items())

dict_items([('FED006', '/VBM/FED006/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141106153429_5.nii'), ('FED007', '/VBM/FED007/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141107104226_5.nii'), ('FED008', '/VBM/FED008/T1_MPRAGE_SAG_ISO_0_9_0005_t1_mprage_sag_iso_0.9_20141112155009_5.nii')])


In [69]:
# perform operations on images using fslmaths
from nipype.interfaces.fsl import ImageMaths


In [70]:
%%bash
# check if operator -mul is even appropriate in this case
fslmaths -h


Usage: fslmaths [-dt <datatype>] <first_input> [operations and inputs] <output> [-odt <datatype>]

Datatype information:
 -dt sets the datatype used internally for calculations (default float for all except double images)
 -odt sets the output datatype ( default is float )
 Possible datatypes are: char short int float double input
 "input" will set the datatype to that of the original image

Binary operations:
  (some inputs can be either an image or a number)
 -add   : add following input to current image
 -sub   : subtract following input from current image
 -mul   : multiply current image by following input
 -div   : divide current image by following input
 -rem   : modulus remainder - divide current image by following input and take remainder
 -mas   : use (following image>0) to mask current image
 -thr   : use following number to threshold current image (zero anything below the number)
 -thrp  : use following percentage (0-100) of ROBUST RANGE to threshold current image (zero any